In [1]:
import json
import keras_tuner as kt
import numpy as np
import pandas as pd
import plotly.express as px
import tensorflow as tf
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from tensorflow import keras
from tensorflow.keras import layers

---
### Data Reading
---

In [2]:
data = np.load('../data/training_data.npz', allow_pickle=True)

train_set = data['train_set']
test_set = data['test_set']
val_set = data['val_set']

In [3]:
def create_dataframe(set):
    df_ = pd.DataFrame([], columns=['x', 'y'])	
    for i in range(len(set)):
        df_ = pd.concat([df_, pd.DataFrame(
            {'reading': [set[i]['x']],
            'posture': set[i]['y']}
        )])
    return df_

In [4]:
df_train = create_dataframe(train_set)
df_test = create_dataframe(test_set)
df_val = create_dataframe(val_set)

print(df_train.shape, df_test.shape, df_val.shape)

(108, 4) (54, 4) (27, 4)


In [5]:
df_train['reading'].iat[0].shape

(24, 32)

In [6]:
X_train = df_train['reading']
y_train = df_train['posture']
X_val = df_val['reading']
y_val = df_val['posture']
X_test = df_test['reading']
y_test = df_test['posture']
# One-hot encode the labels
y_train = pd.get_dummies(y_train).values
y_val = pd.get_dummies(y_val).values
y_test = pd.get_dummies(y_test).values

In [7]:
X_train = np.array([np.array(x) for x in X_train])
X_train = np.expand_dims(X_train, -1)
X_val = np.array([np.array(x) for x in X_val])
X_val = np.expand_dims(X_val, -1)
X_test = np.array([np.array(x) for x in X_test])
X_test = np.expand_dims(X_test, -1)
y_train = np.array([np.array(x) for x in y_train])
y_test = np.array([np.array(x) for x in y_test])
y_val = np.array([np.array(x) for x in y_val])
# Convert the data to tensors
X_train = tf.convert_to_tensor(X_train)
X_val = tf.convert_to_tensor(X_val)
X_test = tf.convert_to_tensor(X_test)
y_train = tf.convert_to_tensor(y_train)
y_val = tf.convert_to_tensor(y_val)
y_test = tf.convert_to_tensor(y_test)

---
### Model Definition
---

In [8]:
def model_builder(hp):
    hp_channels_1 = hp.Int('channels_1', min_value=4, max_value=16, step=4)
    hp_channels_2 = hp.Int('channels_2', min_value=4, max_value=8, step=4)
    hp_dense_1 = hp.Int('dense_1', min_value=16, max_value=64, step=16)
    hp_dense_2 = hp.Int('dense_2', min_value=16, max_value=32, step=16)
    model = keras.Sequential(
        [
            layers.Conv2D(hp_channels_1, kernel_size=(3, 3), activation='relu', padding='same', input_shape=X_train.shape[1:]),
            layers.MaxPooling2D(pool_size=(2, 2)),
            layers.Conv2D(hp_channels_2, kernel_size=(3, 3), activation='relu', padding='same'),
            layers.MaxPooling2D(pool_size=(2, 2)),
            layers.Flatten(),  # Flatten the output of the CNN
            layers.Dense(hp_dense_1, activation='relu'),
            layers.Dense(hp_dense_2, activation='relu'),
            layers.Dense(3, activation='softmax'),
        ]
    )
    model.compile(
        loss='categorical_crossentropy',
        optimizer='adam',
        metrics=['accuracy'],
    )
    return model

In [9]:
early_stopping = keras.callbacks.EarlyStopping(
    patience=20,
    min_delta=0.001,
    restore_best_weights=True,
)
reduce_lr = keras.callbacks.ReduceLROnPlateau(
    patience=5
)

In [10]:
tuner = kt.GridSearch(
    model_builder,
    objective='val_loss',
    max_retries_per_trial=5,
    overwrite=True,
    directory='gridsearch-laguardia-no-DA',
    project_name='posture',
)

---
### Model Training
---

In [11]:
tuner.search(X_train, y_train, validation_data=(X_val, y_val), callbacks=[early_stopping, reduce_lr], epochs=500, batch_size=32)

Trial 64 Complete [00h 00m 02s]
val_accuracy: 0.9629629850387573

Best val_accuracy So Far: 1.0
Total elapsed time: 00h 04m 00s


In [12]:
tuner.results_summary()

Results summary
Results in gridsearch-laguardia-no-DA/posture
Showing 10 best trials
Objective(name="val_accuracy", direction="max")

Trial 0005 summary
Hyperparameters:
channels_1: 4
channels_2: 4
dense_1: 48
dense_2: 32
Score: 1.0

Trial 0014 summary
Hyperparameters:
channels_1: 4
channels_2: 8
dense_1: 64
dense_2: 16
Score: 1.0

Trial 0017 summary
Hyperparameters:
channels_1: 8
channels_2: 4
dense_1: 16
dense_2: 32
Score: 1.0

Trial 0021 summary
Hyperparameters:
channels_1: 8
channels_2: 4
dense_1: 48
dense_2: 32
Score: 1.0

Trial 0031 summary
Hyperparameters:
channels_1: 8
channels_2: 8
dense_1: 64
dense_2: 32
Score: 1.0

Trial 0037 summary
Hyperparameters:
channels_1: 12
channels_2: 4
dense_1: 48
dense_2: 32
Score: 1.0

Trial 0046 summary
Hyperparameters:
channels_1: 12
channels_2: 8
dense_1: 64
dense_2: 16
Score: 1.0

Trial 0060 summary
Hyperparameters:
channels_1: 16
channels_2: 8
dense_1: 48
dense_2: 16
Score: 1.0

Trial 0002 summary
Hyperparameters:
channels_1: 4
channels_2: 4

---
### Model Evaluation
---

In [19]:
tf.get_logger().setLevel('ERROR')
best_models = tuner.get_best_models(num_models=10)

for i, model in enumerate(best_models):
    print(i, model.evaluate(X_test, y_test))

2/2 [==============================] - 0s 4ms/step - loss: 0.9353 - accuracy: 0.7222
0 [0.9352877736091614, 0.7222222089767456]
2/2 [==============================] - 0s 2ms/step - loss: 0.5176 - accuracy: 0.8889
1 [0.5176371335983276, 0.8888888955116272]
2/2 [==============================] - 0s 2ms/step - loss: 0.5796 - accuracy: 0.7222
2 [0.5796219706535339, 0.7222222089767456]
2/2 [==============================] - 0s 3ms/step - loss: 0.8575 - accuracy: 0.7778
3 [0.8574866652488708, 0.7777777910232544]
2/2 [==============================] - 0s 2ms/step - loss: 0.8888 - accuracy: 0.7963
4 [0.8888003826141357, 0.7962962985038757]
2/2 [==============================] - 0s 2ms/step - loss: 0.4332 - accuracy: 0.8704
5 [0.4331575930118561, 0.8703703880310059]
2/2 [==============================] - 0s 2ms/step - loss: 0.5937 - accuracy: 0.7778
6 [0.593673050403595, 0.7777777910232544]
2/2 [==============================] - 0s 2ms/step - loss: 0.4092 - accuracy: 0.8889
7 [0.409153729677200

In [21]:
best_model = tuner.get_best_models(10)[7]
best_model.save('best_cmodel_no_DA.keras')

In [22]:
best_model.evaluate(X_test, y_test)

2/2 [==============================] - 0s 4ms/step - loss: 0.4092 - accuracy: 0.8889


[0.4091537296772003, 0.8888888955116272]